* Author: Gordon Erlebacher
* Date: 2022-03-03
* Purpose: convert Sent field to either date (nb seconds from a fixed time) or an empty field.

In [2]:
import pandas as pd
import numpy as np
import regex as rex
from datetime import datetime
from dateutil import parser
from dateutil.tz import gettz
from unidecode import unidecode
import pytz
import date_library as datelib

In [3]:
parser.parse("Sat Oct 11 17:13:46 UTC 2003")

datetime.datetime(2003, 10, 11, 17, 13, 46, tzinfo=tzutc())

In [5]:
filenm = "output_with_stats_columns.csv.gz"
filenm = "output_0211.csv"
df = pd.read_csv(filenm)

In [6]:

# Standard time :  UTC-5.0 
# Daylight time :  UTC-4.0

tzinfo = {
          "EDT": pytz.timezone('US/Eastern'),
          "CDT": pytz.timezone('US/Central'),
          "MDT": pytz.timezone('US/Mountain'),
          "PDT": pytz.timezone('US/Pacific'), 
          "HDT": pytz.timezone('US/Hawaii'), 
          "GMT": pytz.timezone('UTC'),
    
          # We will convert all times to UTC, then to seconds since 1970
          # Then we will add/subtrace one hour (3600 sec) if time is in 
          # Savings Time (as opposed to Daylight Time)
          "EST": pytz.timezone('US/Eastern'),
          "CST": pytz.timezone('US/Central'),
          "MST": pytz.timezone('US/Mountain'),
          "PST": pytz.timezone('US/Pacific'), 
          "HST": pytz.timezone('US/Hawaii'),  
         }

# For example: when Eastern Daylight Time is used, the international meetings are held 
# between 10 am to 6 pm whereas for Eastern Standard Time the same meeting and conference 
# are held between 9 am to 5 pm.  So, 10 am EDT  <=====> 9 am EST

def normalize(name1, exceptions, date_dict, time_zone):
    # try: 
    # flag = False
    
    try:
        if type(name1) != str:
            date_dict.append((name1, '', ''))
            time_zone.append('')
            return

        if rex.match(r'.*GMT', name1):   # missing some matches. WHY?
            # print(type(name), name)//
            if rex.match(r'.*2\/16\/17 ', name1): flag = True
            name = rex.sub(r'\(GMT-05:00\)', ' EST ', name1)
            name = rex.sub(r'\(GMT-04:00\)', ' EDT ', name)
            name = rex.sub(r'(\d+?\/\d+?)\/(\d\d )', r'\1/20\2', name)  # will fail in the year 21xx <<<< NOT WORKING
        else:
            name = name1

        # Missing 4-digit year
        if not rex.match(r'.*\d{4}', name):
            date_dict.append((name1, '', ''))
            time_zone.append('')
            return

        name = rex.sub(r'(Febnlaiy|Febnlaly|Febiuaiy|Feb1ua1y|Februa ry)', 'February', name)
        name = rex.sub(r'(Janualy|J anuary)', 'January', name)
        name = rex.sub(r"(\'iuesday)", 'Tuesday', name) 
        name = rex.sub(r'Septem ber', 'September', name)

        name = rex.sub(r',(\d{4})', r', \1', name)
        name = rex.sub(r'(\d{4})(\d\d:)', r'\1 \2', name)
        name = rex.sub(r'(\d{4} \d) :', r'\1:', name) 
        name = rex.sub(r'(\d?:\d?:) (\d?)', r'\1\2', name)
        name = rex.sub(r'(\d{4} \d+:) (\d)', r'\1\2', name)  # CHECK ORIGINAL TEXT. Why does this error occur?

        name = rex.sub(r'(\w+ \d+):( \d{4})', r'\1, \2', name)
        name = rex.sub(r'\>', ':', name)
        name = rex.sub(r'\?', '', name)

        tz = 'EST'
        if rex.match('.*Eastern Daylight Time', name):
            name = rex.sub('Eastern Daylight Time', 'EDT', name)  # difference between Daylight and Standard
            tz = 'EDT'
        elif rex.match('.*Eastern Standard Time', name):
            name = rex.sub('Eastern Standard Time', 'EST', name)
            tz = 'EST'
        elif rex.match('.*PST', name):
            tz = 'PST'
        elif rex.match('.*PDT', name):   # Pacific
            tz = 'PDT'
        elif rex.match('.*HST', name):   # Hawaii
            tz = 'HST'
        elif rex.match('.*MDT', name):   # Mountain
            tz = 'MDT'
        elif rex.match('.*CDT', name):   # Central
            tz = 'CDT'
        elif rex.match('.*CST', name):   # Central
            tz = 'CST'
        elif rex.match('.*GMT', name):
            tz = 'GMT'
        else:
            tz = 'EDT'  # default if nothing else

        date_name = parser.parse(name, fuzzy=True, dayfirst=False, tzinfos=tzinfo)

        # t is now a PDT datetime; convert it to UTC
        date_name = date_name.astimezone(pytz.utc)
        date_dict.append((name1, name, date_name))
        time_zone.append(tz)
    except:
        try:
            date_name = parser.parse(name, fuzzy=True, dayfirst=False, tzinfos=tzinfo)
            date_name = name.astimezone(pytz.utc)
            date_dict.append((name1, name, date_name))
            time_zone.append(tz)
        except:
            # Ignore any line with more than 50 characters
            # Ignore any line with the word "the"
            # Ignore any line with two dates
            if rex.match(r'\A.*the.*\Z', name): 
                date_dict.append((name1, '', ''))
                time_zone.append('')
                return
            # Ignore line if the year appears twice
            if rex.match('\A.*(\d{4}).*(\d{4})', name): 
                date_dict.append((name1, '', ''))
                time_zone.append('')
                return
            # Ignore line if there is no number
            if not rex.match('.*\d', name): 
                date_dict.append((name1, '', ''))
                time_zone.append('')
                return
            if len(name) > 40:
                date_dict.append((name1, '', ''))
                time_zone.append('')
                return
            # if there is year in the string
            if not rex.match(r'.*\d{4}', name):
                date_dict.append((name1, '', ''))
                time_zone.append('')
                return
            exceptions.append(name)
            date_dict.append((name1, '', ''))
            time_zone.append('')
                
# Print the date in normalized form so I can spot check. 
# Then save them to a file. 
# On any date that is not valid, make it empty, tag the row and remove it from the output.csv file. 

In [7]:
tzinfo

{'EDT': <DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>,
 'CDT': <DstTzInfo 'US/Central' LMT-1 day, 18:09:00 STD>,
 'MDT': <DstTzInfo 'US/Mountain' LMT-1 day, 17:00:00 STD>,
 'PDT': <DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>,
 'HDT': <DstTzInfo 'US/Hawaii' LMT-1 day, 13:29:00 STD>,
 'GMT': <UTC>,
 'EST': <DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>,
 'CST': <DstTzInfo 'US/Central' LMT-1 day, 18:09:00 STD>,
 'MST': <DstTzInfo 'US/Mountain' LMT-1 day, 17:00:00 STD>,
 'PST': <DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>,
 'HST': <DstTzInfo 'US/Hawaii' LMT-1 day, 13:29:00 STD>}

In [8]:
sent_lst = df['Sent'].values
len(sent_lst)

71143

In [9]:
name = '2/16/17 3:25 PM (GMT-05:00)'
name = rex.sub(r'(\d?\/\d)\/(\d\d )', r'\1/20\2', name)
rex.match(r'(\d+?\/\d+?)\/(\d\d )', name)

<regex.Match object; span=(0, 8), match='2/16/17 '>

In [10]:
exceptions = []
date_dict = []
time_zone = []
for sent in sent_lst:
    normalize(sent, exceptions, date_dict, time_zone)
    
len(exceptions), len(sent_lst), df.shape

/Users/erlebach/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname US identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Users/erlebach/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PMEDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Users/erlebach/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname OS identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an excepti

(1, 71143, (71143, 15))

In [11]:
exceptions

[' September 25, 2014 at 12:42: 22 PM EDT']

* 'Friday, May 13, 201611:18 AM': datetime.datetime(2022, 5, 13, 18, 16, 11),
* 

In [12]:
dates_orig = []
dates_new = []
dates_date = []
date_adj = []
timestamp = []
for i, (tz_el, el) in enumerate(zip(time_zone, date_dict)):
    dates_orig.append(el[0])
    dates_new.append(el[1])
    dates_date.append(el[2])
    try:
        # subtract 5 hours to convert back to Tallahassee time
 
        if False:  # take daylight savings properly into account
            if tz_el[1] == 'S':  # adjust time if Savings
                timestp -= 3600
        else:  
            timestp = el[2].timestamp() - 5 * 3600 
            
        timestamp.append(timestp)
        # Transform time from timestamp back to UTC
        dtime = datelib.timestampToDateTimeUTC(timestp)
        date_adj.append(dtime)
    except:
        timestamp.append(-1)
        date_adj.append(('',''))
    
df1 = pd.DataFrame({'orig':dates_orig, 'new':dates_new, 'date':dates_date, 'TZ': time_zone, 'date_adj': date_adj, 'timestamp': timestamp})
print(len(date_dict), len(dates_orig), df.shape)

df1.to_csv("dates.csv", index=0)

71143 71143 (71143, 15)


In [13]:
df.shape, df1.shape

((71143, 15), (71143, 6))

In [14]:
# Add new columns: new date and time, and number of seconds since 1970
# timestamp: seconds since 1970
# dates_orig: original send column
# date_adj[0]: adjusted date
# date_adj[1]: adjusted time
adj_date = []
adj_time = []
for dat_tim in date_adj:
    adj_date.append(dat_tim[0])
    adj_time.append(dat_tim[1])
    
df['timestamp'] = timestamp
df['date_sent'] = adj_date
df['time_sent'] = adj_time
df.columns

outfile = "output_stats_timestamps.csv.gz"
outfile = "output_03112022.csv"
df.to_csv(outfile, index=0)

In [590]:
timestamp;
df.columns

Index(['filenm', 'From', 'Sent', 'To', 'CC', 'Bcc', 'Subject', 'Attachments',
       'Importance', 'isThread', 'isAutoMessage', 'isDisplacement',
       'hasAllCapLine', 'hasBadDate', 'Body', 'nb_words', 'nb_chars',
       'body_len', 'body', 'Error_from', 'Error_sent', 'mn_nb_words',
       'std_nb_words', 'mn_nb_chars', 'std_nb_chars', 'count', 'email_count',
       'timestamp', 'date_sent', 'time_sent'],
      dtype='object')